<a href="https://colab.research.google.com/github/davidbau/colabsecrets/blob/master/notebooks/colabnotebooksecrets.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# What makes Colab so great?

In [ ]:
%%bash
nvidia-smi

Yes, that's right.  Every Colab user gets their very own GPU, complements of Google!

The main disadvantage of colab it might not come with the exact environment you are used to.  But there are two reasons this is not really a problem.

1. The defaults are pretty reasonable.
2. You are root.

Check out the defaults: they include the latest versions of pytorch, tensorflow, java, gcc, etc.

In [ ]:
%%bash

# As of this writing, colab gives us an Ubuntu 18.04 machine.
lsb_release -a

# It includes a whole pile of software, e.g., perl, python, java, gcc, git, etc
apt list --installed

# And it preinstalls a healthy set of python: pytorch, tensorflow, pandas, etc.
pip list

And then check out the root access. You can install new packages at will.

In [ ]:
%%bash
whoami

In [ ]:
%%bash
apt-get install -q blender

# Colab Notebook Boostrapping

The main Google Colab disadvantage is that your data and environment are not there yet. So to make a Colab notebook, here is our plan of attack.
1. Make a standard Jupyter notebook in your own project, on your own machine.
2. Then make the notebook bring in its own dependencies when running on colab.

Once we have a working notebook, at the top all we need to do is include a small section that runs only if the notebook is running on Colab.  It can look something like this:



In [ ]:
%%bash

# Exit this cell if not running in colab.
!(stat -t /usr/local/lib/*/dist-packages/google/colab > /dev/null 2>&1) && exit 
pip install ninja 2>> install.log
git clone https://github.com/davidbau/colabsecrets.git tutorial_code 2>> install.log

In [ ]:
try: # this will alter the path if inside Google colab.
    import google.colab, sys, torch
    sys.path.append('/content/tutorial_code')
    if not torch.cuda.is_available():
        print("Change runtime type to include a GPU.")  
except:
    pass

# Loading and Saving Data

There are two options to load data into Google Colab. You can either download it directly from a server using a url, or you can load it by connecting to your Google Drive.

The first option is easier and will save you Drive space. However, the data will not be persistent when you close the colab session.

In [ ]:
import requests

# Download a file into the filename fn
def download(url, fn=None):
  if fn is None:
    fn = url.split('/')[-1]
  r = requests.get(url)
  if r.status_code == 200:
      open(fn, 'wb').write(r.content)
      print("{} downloaded: {:.2f} KB".format(fn, len(r.content)/1024.))
  else:
      print("url not found:", url)

In [ ]:
download('http://6.869.csail.mit.edu/fa19/psets19/pset6/WelshCorgi.jpeg', 'dog.jpeg')
! ls .

To use Google Drive, you need to run the following code, and enter the authorization code.

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

You can access your data at `/content/drive/My Drive/`

# Making direct links to notebooks on github

You can make a direct link to a notebook on github using URLs of the following form:

If tbe notebook can be seen on github with the following URL:
    
`https://github.com/davidbau/colabsecrets/blob/master/notebooks/colabnotebooksecrets.ipynb`

Then it can be opened on colab with the following URL:
    
`https://colab.research.google.com/github/davidbau/colabsecrets/blob/master/notebooks/colabnotebooksecrets.ipynb`

# Other Google Colab Survival Tricks

1. Use torch.hub to download models and automatically cache them, so you do not need to redownload things.
2. Use a utility like `netdissect.show` for showing grids of images and other data.
3. Unlike juypter, every Colab cell runs on its own independent webpage (in an iframe), so normal widgets do not work.  Use `netdissect.labwidgets` instead.

See the following examples to show these techniques.

https://colab.research.google.com/github/SIDN-IAP/global-model-repr/blob/master/notebooks/gandissect_solutions.ipynb

https://colab.research.google.com/github/SIDN-IAP/interactivity/blob/master/notebooks/ganpaint-solution.ipynb

# Working with Images in Colab

downloading and loading a dataset (see settings.py)

In [ ]:
from experiment import setting
dsv = setting.load_dataset('places', 'val')

Showing an image

In [ ]:
from netdissect import imgviz
iv = imgviz.ImageVisualizer(128, source=dsv)

In [ ]:
iv.image(dsv[10000][0])

Showing a lot of images

In [ ]:
from netdissect import show
show([[i, iv.image(dsv[i][0])] for i in range(14)])

# Interactive Widgets in Colab

Make some interactions using labwidgets.

A labwidget is just a piece of HTML that you can put on the notebook.

In [ ]:
some_input = labwidget.Textbox('123-132')
btn = labwidget.Button('click me')
div = labwidget.Div('<h1>My Title</h1><p>This is a paragraph of some HTML text.</p>')

show(some_input)
show(btn)
show(div)

You can change a labwidget's values and it will change on the screen right away.  You can also read values from the widgets.

In [ ]:
div.innerHTML = '<h2>Some new text</h2>'
btn.style = {'width': '100%', 'background': 'pink'}
print(some_input.value)

Finally, the fancy thing you can do is hook up functions to the widgets that are called when they are clicked.

In [ ]:
def show_range():
    if '-' in some_input.value:
        first, last = [int (i) for i in some_input.value.split('-', 1)]
        values = list(range(first, last + 1))
    else:
        values = [int(some_input.value)]
    div.show([[i, iv.image(dsv[i][0])] for i in values])
btn.on('click', show_range)

You can bundle together widgets into your own widgets and make more complicated stuff.

In [ ]:
from netdissect import labwidget

class MyWidget(labwidget.Widget):
    def __init__(self):
        super().__init__()
        self.inp = labwidget.Textbox()
        self.img = labwidget.Image()
        def changed_value():
            i = int(self.inp.value)
            self.img.render(iv.image(dsv[i][0]))
        self.inp.on('value', changed_value)
    def widget_html(self):
        return show.html([self.inp, self.img])

widget1 = MyWidget()
widget2 = MyWidget()
show([widget1, widget2])